# Smart Spin-Locking with waiting

In [3]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [4]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')   
try:
    pulsedmeasurementlogic
except NameError:
    manager.startModule('logic', 'pulsedmeasurementlogic')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [5]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [6]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV2'
setup['poi'] = poimanagerlogic.active_poi.get_key()
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 10000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [7]:
# for single frequency XY8-N:
HHSW = OrderedDict()
HHSW['mw_amp'] = 0.25
HHSW['spinlock_amp'] = 0.122
HHSW['mw_freq'] = 100e6
HHSW['tau_start'] = 10.0*1e-9
HHSW['tau_incr'] = 5.0*1e-6
HHSW['points'] = 40
HHSW['lock_time'] = 117.0*1e-6
HHSW['alternating']=True
HHSW['rabi_period']=188.5*1e-9
HHSW['sweeps']=1000000

In [8]:
tau = HHSW['tau_start'] + np.arange(HHSW['points']) * HHSW['tau_incr']

start_time = time.time()
for t in tau:
    generate_sequence(t)
    time.sleep(4)
    refocus_poi(setup['poi'])
    time.sleep(5)
    end_measure=True
    end_measure = do_smart_hh_with_waiting(save_tag='',load_tag='', tau=np.array([t]))
    if end_measure:
        time.sleep(1)
    time.sleep(2)
#     refocus_poi(setup['poi'])
#     time.sleep(5)
    print((time.time() - start_time)/60)    

381370.0 2.692419268011091e-09
4.777154902617137
381370.0 5.318588567539031e-10
9.717608257134755
381370.0 5.4087455274802855e-09
14.830797290802002
381370.0 5.624874136481378e-10
20.071531081199645
381370.0 1.9018257868335612e-11
25.485799884796144
381370.0 1.469006674658692e-10
31.057674904664356
381370.0 1.2775614774472675e-08
36.82163327932358
381370.0 1.0316991016587492e-09
42.73691695133845
381370.0 1.1743859719193011e-09
48.801826270421344
381370.0 4.6957807064014175e-09
55.041483322779335
381370.0 3.584568302841579e-09
61.47147598663966
381370.0 3.177851085793895e-09
68.05177952845891
381370.0 2.6600013285099254e-09
74.80029158592224
381370.0 3.079023352730789e-08
81.72007484038672
381370.0 2.216641450247255e-08
88.79271869659424
381370.0 2.097295740071894e-08
96.04969245592753
381370.0 3.724949055260683e-08
103.45641049544017
381370.0 1.0310699478035417e-08
111.02733273903529
381370.0 2.07079606293325e-08
118.7580335299174
381370.0 2.193884976814999e-08
126.67225805521011
3813

In [1]:
def generate_sequence(tau):

    sequencegeneratorlogic.delete_sequence('HHSMWAIT')    
    sequencegeneratorlogic.generate_HH_SMART_waiting('HHSMWAIT', HHSW['rabi_period'], HHSW['spinlock_amp'], 100e6,
                       0.25, tau, HHSW['lock_time'], 'a_ch1', 3.0e-6, 2.0, 0.7e-6, 1.0e-6, '', 'd_ch2', True)
    # sample, upload and load waveform
    if 'HH_waiting' not in sequencegeneratorlogic.saved_pulse_sequences:
        sequencegeneratorlogic.log.error('Sequence generation of HHSMWAIT in notebook timed out.')
    pulsedmasterlogic.sample_sequence('HHSMWAIT', True)

    while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
            time.sleep(1)

In [2]:
def refocus_poi(poi=None, max_drift=0.3,min_counts=setup["min_counts"]):
    if poi==None:
        return False
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.module_state() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.module_state() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_smart_hh_with_waiting(save_tag='',load_tag='', tau=np.array([1.0*1e-6])):
    
    pulsedmasterlogic.do_fit('No Fit')
    # set parameters in analysis tab
    if HHSW['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau, 2, 100e-6, [], HHSW['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau, 2, 100e-6, [], HHSW['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.21e-6)
    pulsedmasterlogic.analysis_interval_changed(1)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    
    while pulsedmeasurementlogic.elapsed_sweeps < HHSW['sweeps']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s','HHSW-'+str(HHSW['spinlock_amp'])+'_'+str(tau)+'_'+setup['NV_name'], True)
    time.sleep(2)
    return user_terminated